# BOTPT Data detide for Central Caldera Station E; Chadwick Method

We detide seafloor pressure measurements taken at The Axial Seamount's Central Caldera (RS03CCAL)  

In [ ]:
%reset -f

In [ ]:
# Load map of Axial caldera showing locations of BOTPT instruments (red circles)
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-caldera-ed-sm.png")

In [ ]:
#Zoomed in map of Axial caldera
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-zoom-ed-sm.png")

### API Information Setup

In [ ]:
USERNAME = 'OOIAPI-J97520T1AYPUNI'
TOKEN =  'TEMP-TOKEN-YD01XSNDIO57MP'

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc

#### For mad help with Pandas https://chrisalbon.com
#### For all sorts of great repos https://github.com/rabernat/research_computing.git

#### Read Tide Data E

In [ ]:
tide_file = '/home/jovyan/data/tide_data/formatted_pred/pred_E.txt'
df_tides_E = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides_E['datetime']=df_tides_E['year'] + '-' + df_tides_E['month'] + '-' + df_tides_E['day'] + \
            'T' + df_tides_E['hour'] + ':' + df_tides_E['minute'] + ':' + df_tides_E['second']
#df_tides_E = pd.DataFrame(tides_E)
df_tides_E.index=pd.to_datetime(df_tides_E['datetime'].values)
#df_tides_E.index = df_tides_E['parsed_time']
del df_tides_E['year']
del df_tides_E['month']
del df_tides_E['day']
del df_tides_E['hour']
del df_tides_E['minute']
del df_tides_E['second']
del df_tides_E['datetime']
df_tides_E['height'] = df_tides_E['height'].astype(float)
#df_tides_E['height'] = df_tides_E['height'].to_pandas().resample('T').mean()

# columnsTitles=["datetime","height"]
# df_tides_E=df_tides_E.reindex(columns=columnsTitles)
df_tides_E.head()

In [ ]:
#list comprehention
epoch= [i.timestamp() for i in df_tides_E.index.to_pydatetime()]

In [ ]:
df_tides_E['epoch'] = epoch
df_tides_E.head()

#### Read Downsampled BOTPT Data

In [ ]:
with open('/home/jovyan/data/bottom_pressure15S_E.pd', 'rb') as E:
    botpt_data = pk.load(E)
df_botptE = pd.DataFrame(botpt_data)
df_botptE['bottom_pressure'] = df_botptE['bottom_pressure'].astype(float)
df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptE.index.to_pydatetime()]
df_botptE['epoch'] = epoch
df_botptE.head()

In [ ]:
df_botptE.tail()

#### Interpolate BOTPT onto Tides 

In [ ]:
#df_tides_E['depth'] = np.interp(df_tides_E.epoch.values, df_botptE.epoch.values, df_botptE.depth.values)
#df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670

#### Combine BOTPT with Tide Predictions

In [ ]:
df_subset = df_tides_E.loc['2017-08-01 00:00:00':'2018-05-04 11:59:00']
df_botptE['tides'] = df_subset.height 
df_botptE.head()

In [ ]:
df_botptE['dtide']=  df_botptE['depth'] - df_botptE['tides']
df_botptE.head()

In [ ]:
#df_tides_E.height.plot()
# df_tides_E.sel(date=slice('2015-10-01', '2016-10-01')).plot()
# df_tides_E['5/3/2014':'5/4/2015'].plot()

In [ ]:
time = list(df_botptE.index.values)
height = list(df_botptE['dtide'].values)

time_int = []
time = list(pd.to_datetime(df_botptE.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.hexbin(time_int, height, vmin=0, vmax=30, gridsize=(1500,100), mincnt=1, cmap='Greens', linewidths=0)
fig.colorbar(hb1, pad = 0.01)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 8, 1, 0, 0),datetime.datetime(2017, 12, 1, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()